In [2]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
# When prompted, paste the HF access token you created earlier.
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your 

In [10]:
from datetime import datetime
# Hyperparameters
max_length = 512 # This was an appropriate max length for my dataset
max_steps = 1000 # [500, 1000, 10000]

# Data
storage_dir = '/media/Blue2TB3/jpei'
project = "vox-finetune"
base_model_name = "llama-2-7b-chat"
dataset_names = [
    'teach', 
    'gpt_teacher', 
    'gpt4tools',
    'camel'
]
data_name = '-'.join(dataset_names)
teach_data_dir = f"{storage_dir}/teach-dataset/edh_instances"

# Model
# base_model_id = "meta-llama/Llama-2-7b-hf"
base_model_id = f'{storage_dir}/transformer_data/{base_model_name}' # local model dir: /media/PampusData/jpei/transformer_data/llama-2-7b-chat
timestamp_str = datetime.now().strftime('%Y-%m-%d-%H-%M')
run_name = f'{base_model_name}-{data_name}-{timestamp_str}'
ft_model_id = f'{storage_dir}/{project}/{run_name}' # /media/PampusData/vox-finetune/llama-2-7b-chat
checkpoint_name = f'checkpoint-{max_steps}'
ft_ckpt_id = f'{ft_model_id}/{checkpoint_name}'

In [ ]:
from datasets import load_dataset, DatasetDict
# teach_data_dir = "/media/PampusData/jpei/teach-dataset/edh_instances"
train_dataset = load_dataset('json', data_files=f'{teach_data_dir}/teach_edh_train.jsonl', split='train')  
eval_dataset = load_dataset('json', data_files=f'{teach_data_dir}/teach_edh_valid.jsonl', split='train')
test_dataset = load_dataset('json', data_files=f'{teach_data_dir}/teach_edh_test.jsonl', split='train')
teach_dataset = DatasetDict({"train":train_dataset, "validation": eval_dataset,"test":test_dataset})
!huggingface-cli login
teach_dataset.push_to_hub("Jiahuan/teach_edh", private=True)
teach_dataset = load_dataset("Jiahuan/teach_edh")